### 3. Analýza známek u studijní skupiny (gql_ug + gql_granting).

Jako parametr vezme definici kontingenční tabulky, GQL dotaz, dotaz zrealizuje, a data vloží do sešitu Excel spolu s kontingenční tabulkou. Vytvořte jako FastAPI službu.

Společné podmínky
- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

In [8]:
# saved for later :)

with open('fake_data.json', 'r') as file:
    fake_data = json.load(file)
    
grouped_data = filtered_data + fake_data

print(grouped_data)

with open("grouped_data.json", "w") as json_file:
    json.dump(grouped_data, json_file)

[{'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'level_date': '2023-04-19T08:00:00', 'student_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'student_email': 'john.newbie@world.com', 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programovani', 'semestr_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '6d5a1c9e-3db6-11ec-8d3d-0242ac130003', 'level_name': 'A', 'level_order': 3, 'level_date': '2023-05-20T09:30:00', 'student_id': '4f8e72aa-3db6-11ec-8d3d-0242ac130003', 'student_email': 'emma.smith@example.com', 'subject_id': 'ce250a69-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Pocitacove site', 'semester_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '7e6b3f8a-3db6-11ec-8d3d-0242ac130003', 'level_name': 'B', 'level_order': 4, 'level_date': '2023-06-10T14:00:00', 'student_id': '5fbc249e-3db6-11

</br>

</br>
</br>

### Autentizace

In [23]:
import aiohttp
import asyncio
import json

In [1]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3" #"http://host.docker.internal:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)
        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)

def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql" #"http://host.docker.internal:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

In [2]:
# used different approach

from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)

    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

In [30]:
import asyncio

username = "john.newbie@world.com"
password = "john.newbie@world.com"


### TODO:

queryStr = """
{
  result: groupById(id: "2d9dcd22-a4a2-11ed-b9df-0242ac120003") {
    name
    id
    memberships {
      user {
        id
        fullname
        classifications {
          level {
            id
            name
          }
          id
          order
          semester {
            id
            order
            subject {
              id
              name
            }
          }
        }
      }
    }
  }
}
"""

import pandas as pd
def toTable(data):
    return pd.DataFrame(data)

async def fullPipe():
    global pandasData
    
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})
    
    data = response.get("data", None)
    result = data.get("result", None)
    #print(result)

    def convertLevel(membership):
        user = membership["user"]
        classifications = user["classifications"]

        rows = []
        for classification in classifications:
            row = {}
            row["user_fullname"] = membership["user"]["fullname"]
            row["user_id"] = user["id"]

            row["level_id"] = classification["level"]["id"]
            row["level_name"] = classification["level"]["name"]
            row["level_order"] = classification["order"]

            row["subject_id"] = classification["semester"]["subject"]["id"]
            row["subject_name"] = classification["semester"]["subject"]["name"]

            row["semester_id"] = classification["semester"]["id"]
            row["semester_order"] = classification["semester"]["order"]
            rows.append(row)    
        return rows
    
    
    memberships = result["memberships"]
    converted_data = [convertLevel(membership) for membership in memberships]
    
    flattened_data = [item for sublist in converted_data for item in sublist]

    sorted_data = sorted(flattened_data, key=lambda x: x['level_order'], reverse=True)

    filtered_data = sorted_data[:1]
    
    with open('result.json', 'w') as f:
        json.dump(filtered_data, f)
    
    return print(filtered_data)


await fullPipe()

[{'user_fullname': 'John Newbie', 'user_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programování', 'semester_id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'semester_order': 1}]


In [34]:
with open('result.json', 'r') as file:
    result = json.load(file)
    
####################################
#
# TODO: group with fake data
#
###################################

import pandas as pd
from IPython.display import display

df = pd.DataFrame(result)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

display(df)

,user_fullname,user_id,level_id,level_name,level_order,subject_id,subject_name,semester_id,semester_order
0,John Newbie,2d9dc5ca-a4a2-11ed-b9df-0242ac120003,5faea21a-b095-11ed-9bd8-0242ac110002,C,2,ce250a68-b095-11ed-9bd8-0242ac110002,Programování,ce250af4-b095-11ed-9bd8-0242ac110002,1
